In [2]:
import pandas as pd

# Read all CSV files
csv1 = pd.read_csv('chunk_1.csv')  # Keep header
csv2 = pd.read_csv('chunk_2.csv')  # Read with header
csv3 = pd.read_csv('chunk_3.csv')  # Read with header

# Drop the first row from csv2 and csv3
csv2 = csv2.iloc[1:]
csv3 = csv3.iloc[1:]

/var/folders/lq/vxt5qp993bdgvld4cy_f12qc0000gn/T/ipykernel_88723/4167656697.py:4: DtypeWarning: Columns (3,10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  csv1 = pd.read_csv('chunk_1.csv')  # Keep header
/var/folders/lq/vxt5qp993bdgvld4cy_f12qc0000gn/T/ipykernel_88723/4167656697.py:5: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  csv2 = pd.read_csv('chunk_2.csv')  # Read with header
/var/folders/lq/vxt5qp993bdgvld4cy_f12qc0000gn/T/ipykernel_88723/4167656697.py:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  csv3 = pd.read_csv('chunk_3.csv')  # Read with header


In [3]:
# Create merged df
df = pd.concat([csv1, csv2, csv3], ignore_index=True)

In [4]:
# Check initial columns
# print(df.dtypes)

In [5]:
# Provide summary info on initial columns
import numpy as np

def summary(df):
    table = pd.DataFrame(index=df.columns, columns=['Dtype', '#NA','%NA', '#unique'])
    table['Dtype'] = df.dtypes.values
    table['#NA'] = df.isna().sum().values
    table['%NA'] = np.round((df.isna().sum().values / len(df) * 100), 1)
    table['#unique'] = df.nunique().values
    
    return table

print(summary(df))

                             Dtype      #NA   %NA  #unique
OBJECTID                     int64        0   0.0  2303564
FPA_ID                      object        0   0.0  2303564
NWCG_REPORTING_UNIT_NAME    object        0   0.0     2105
FIRE_NAME                   object   995414  43.2   701431
FIRE_YEAR                    int64        0   0.0       29
DISCOVERY_DATE              object        0   0.0    10593
DISCOVERY_DOY                int64        0   0.0      366
NWCG_CAUSE_CLASSIFICATION   object        0   0.0        3
NWCG_GENERAL_CAUSE          object        0   0.0       13
NWCG_CAUSE_AGE_CATEGORY     object  2228037  96.7        1
CONT_DATE                   object   894813  38.8    10596
CONT_DOY                   float64   894813  38.8      366
FIRE_SIZE                  float64        0   0.0    18539
FIRE_SIZE_CLASS             object        0   0.0        7
LATITUDE                   float64        0   0.0  1153010
LONGITUDE                  float64        0   0.0  12823

In [6]:
# Drop columns
# df = df.drop(df.filter(like='_DOY').columns, axis=1)

# Drop CONT_DOY and AGE column
df = df.drop('CONT_DOY', axis=1)
df = df.drop('NWCG_CAUSE_AGE_CATEGORY', axis=1)

In [7]:
# Convert _DATE columns to datetime
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'])
df['DISCOVERY_DATE'] = df['DISCOVERY_DATE'].dt.tz_localize(None)
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'], errors='coerce')
df['CONT_DATE'] = df['CONT_DATE'].dt.tz_localize(None)
print(df['CONT_DATE'].dtype)

# Convert float to integer, of form that accepts nulls
df['FIPS_CODE'] = df['FIPS_CODE'].astype('Int64')

datetime64[ns]


In [8]:
# Identify columns with mixed dtypes
for col in df.columns:
    if df[col].apply(type).nunique() > 1:
        print(f"Column '{col}' has mixed types.")

Column 'FIRE_NAME' has mixed types.
Column 'CONT_DATE' has mixed types.
Column 'FIPS_NAME' has mixed types.


In [9]:
# For strings of mixed type, force string type
df['FIRE_NAME'] = df['FIRE_NAME'].astype(str)
df['FIPS_NAME'] = df['FIPS_NAME'].astype(str)

In [10]:
# Check the form that nulls take in CONT_DATE
print(df[df['CONT_DATE'].isnull()].head(3))

      OBJECTID      FPA_ID    NWCG_REPORTING_UNIT_NAME       FIRE_NAME  \
84          85  FS-1419068    Kootenai National Forest           HWY 2   
1506      1507  FS-1422198  Mark Twain National Forest  PARTY JUNCTION   
1507      1508  FS-1422199  Mark Twain National Forest  BROWN BOTTLE 2   

      FIRE_YEAR DISCOVERY_DATE  DISCOVERY_DOY NWCG_CAUSE_CLASSIFICATION  \
84         2005     2005-07-04            185                     Human   
1506       2005     2005-03-11             70                     Human   
1507       2005     2005-03-11             70                     Human   

           NWCG_GENERAL_CAUSE CONT_DATE  FIRE_SIZE FIRE_SIZE_CLASS   LATITUDE  \
84                  Fireworks       NaT        0.3               B  48.228056   
1506  Recreation and ceremony       NaT        0.1               A  37.029722   
1507       Arson/incendiarism       NaT        0.1               A  36.829722   

       LONGITUDE STATE  FIPS_CODE       FIPS_NAME  
84   -115.480278    MT   

In [11]:
# For cont dates with nulls, set them to a datetime version of null
if pd.api.types.is_datetime64_any_dtype(df['CONT_DATE']):
    df['CONT_DATE'] = df['CONT_DATE'].dt.tz_localize(None)

df['CONT_DATE'] = df['CONT_DATE'].fillna(pd.Timestamp('1700-01-01'))



In [12]:
# Check no more columns with mixed dtypes
for col in df.columns:
    if df[col].apply(type).nunique() > 1:
        print(f"Column '{col}' has mixed types.")

In [13]:
# Calculate MONTH
df['MONTH']= df['DISCOVERY_DATE'].dt.strftime('%b')

In [14]:
print(df.dtypes)

OBJECTID                              int64
FPA_ID                               object
NWCG_REPORTING_UNIT_NAME             object
FIRE_NAME                            object
FIRE_YEAR                             int64
DISCOVERY_DATE               datetime64[ns]
DISCOVERY_DOY                         int64
NWCG_CAUSE_CLASSIFICATION            object
NWCG_GENERAL_CAUSE                   object
CONT_DATE                    datetime64[ns]
FIRE_SIZE                           float64
FIRE_SIZE_CLASS                      object
LATITUDE                            float64
LONGITUDE                           float64
STATE                                object
FIPS_CODE                             Int64
FIPS_NAME                            object
MONTH                                object
dtype: object


In [15]:
# Calculate DURATION

# idea: df['DURATION'] = (df['CONT_DATE'] - df['DISCOVERY_DATE']) + 1
# missing DURATIONS forced to take value of 0, so type consistent

def calculate_duration(row):
    if row['CONT_DATE'] == pd.Timestamp('1700-01-01'):
        return 0
    else:
        return (row['CONT_DATE'] - row['DISCOVERY_DATE']).days + 1

df['DURATION'] = df.apply(calculate_duration, axis=1)

# Display values and counts of DURATION; 0 values are placeholders for nulls
# print(df['DURATION'].value_counts())

In [16]:
# To make reordering columns in next step easier
print(df.columns.tolist())

['OBJECTID', 'FPA_ID', 'NWCG_REPORTING_UNIT_NAME', 'FIRE_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE', 'CONT_DATE', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'STATE', 'FIPS_CODE', 'FIPS_NAME', 'MONTH', 'DURATION']


In [17]:
# Reorder the DataFrame
newOrder = ['OBJECTID', 'FPA_ID', 'NWCG_REPORTING_UNIT_NAME', 'FIRE_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'CONT_DATE', 'DURATION', 'MONTH', 'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'STATE', 'FIPS_CODE', 'FIPS_NAME']

df = df[newOrder]

In [18]:
df.head()

,OBJECTID,FPA_ID,NWCG_REPORTING_UNIT_NAME,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,CONT_DATE,DURATION,MONTH,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,FIPS_CODE,FIPS_NAME
0,1,FS-1418826,Plumas National Forest,FOUNTAIN,2005,2005-02-02,2005-02-02,1,Feb,Human,Power generation/transmission/distribution,0.10,A,40.036944,-121.005833,CA,6063,Plumas County
1,2,FS-1418827,Eldorado National Forest,PIGEON,2004,2004-05-12,2004-05-12,1,May,Natural,Natural,0.25,A,38.933056,-120.404444,CA,6061,Placer County
2,3,FS-1418835,Eldorado National Forest,SLACK,2004,2004-05-31,2004-05-31,1,May,Human,Debris and open burning,0.10,A,38.984167,-120.735556,CA,6017,El Dorado County
3,4,FS-1418845,Eldorado National Forest,DEER,2004,2004-06-28,2004-07-03,6,Jun,Natural,Natural,0.10,A,38.559167,-119.913333,CA,6003,Alpine County
4,5,FS-1418847,Eldorado National Forest,STEVENOT,2004,2004-06-28,2004-07-03,6,Jun,Natural,Natural,0.10,A,38.559167,-119.933056,CA,6003,Alpine County


In [19]:
# List of unique values for general causes
print("General causes:")
for cause in df['NWCG_GENERAL_CAUSE'].unique():
    print(cause)

print()

# List of unique values for cause classifications
print("Cause classifications:")
for causeClass in df['NWCG_CAUSE_CLASSIFICATION'].unique():
    print(causeClass)

General causes:
Power generation/transmission/distribution
Natural
Debris and open burning
Missing data/not specified/undetermined
Recreation and ceremony
Equipment and vehicle use
Arson/incendiarism
Fireworks
Other causes
Railroad operations and maintenance
Smoking
Misuse of fire by a minor
Firearms and explosives use

Cause classifications:
Human
Natural
Missing data/not specified/undetermined


In [20]:
df['NWCG_GENERAL_CAUSE'] = df['NWCG_GENERAL_CAUSE'].replace('Missing data/not specified/undetermined', 'Not specified')
df['NWCG_CAUSE_CLASSIFICATION'] = df['NWCG_CAUSE_CLASSIFICATION'].replace('Missing data/not specified/undetermined', 'Not specified')

In [22]:
# Group by the FIRE_SIZE_CLASS column and show number of fires, ave duration (ave duration drops obs with missing/0 values), and total burned acres

num_fires_by_size = df.groupby('FIRE_SIZE_CLASS').size().reset_index(name='NUM_FIRES')

# Calculate average duration by fire size class, excluding 0 durations
avg_duration_by_size = df[df['DURATION'] != 0].groupby('FIRE_SIZE_CLASS')['DURATION'].mean().reset_index(name='AVG_DURATION')

# Aggregate total fire size by fire size class
total_size_by_class = df.groupby('FIRE_SIZE_CLASS')['FIRE_SIZE'].sum().div(1000).reset_index(name='TOTAL BURN ACRES (000s)')

# Merge all DataFrames
num_fires_by_size = pd.merge(num_fires_by_size, avg_duration_by_size, on='FIRE_SIZE_CLASS')
num_fires_by_size = pd.merge(num_fires_by_size, total_size_by_class, on='FIRE_SIZE_CLASS')

# Sort by NUM_FIRES in descending order
num_fires_by_size = num_fires_by_size.sort_values(by='NUM_FIRES', ascending=False).reset_index(drop=True)

print(num_fires_by_size)

fire_size_classes = {
    "Class A": "one-fourth acre or less",
    "Class B": "more than one-fourth acre, but less than 10 acres",
    "Class C": "10 acres or more, but less than 100 acres",
    "Class D": "100 acres or more, but less than 300 acres",
    "Class E": "300 acres or more, but less than 1,000 acres",
    "Class F": "1,000 acres or more, but less than 5,000 acres",
    "Class G": "5,000 acres or more"
}
print()
for key, value in fire_size_classes.items():
    print(f"{key}: {value}")

  FIRE_SIZE_CLASS  NUM_FIRES  AVG_DURATION  TOTAL BURN ACRES (000s)
0               B    1104386      1.534061              2337.794412
1               A     876408      1.682341               100.666470
2               C     257096      2.134544              7367.032949
3               D      34008      4.461120              5516.780005
4               E      17191      7.904617              8860.616902
5               F       9692     14.845928             20733.970301
6               G       4783     33.028926            135131.873373

Class A: one-fourth acre or less
Class B: more than one-fourth acre, but less than 10 acres
Class C: 10 acres or more, but less than 100 acres
Class D: 100 acres or more, but less than 300 acres
Class E: 300 acres or more, but less than 1,000 acres
Class F: 1,000 acres or more, but less than 5,000 acres
Class G: 5,000 acres or more


In [23]:
# Group by FIRE_YEAR and show number of fires, ave duration (ave duration drops obs with missing/0 values), and total burned acres
num_fires_by_year = df.groupby('FIRE_YEAR').size().reset_index(name='NUM_FIRES')

# Calculate the average duration per year, excluding 0 durations
avg_duration_by_year = df[df['DURATION'] != 0].groupby('FIRE_YEAR')['DURATION'].mean().reset_index(name='AVG_DURATION')

# Aggregate total fire size by year and divide by 1000
total_size_by_year = df.groupby('FIRE_YEAR')['FIRE_SIZE'].sum().div(1000).reset_index(name='TOTAL BURN ACRES (000s)')

# Merge the DataFrames on FIRE_YEAR
num_fires_by_year = pd.merge(num_fires_by_year, avg_duration_by_year, on='FIRE_YEAR')
num_fires_by_year = pd.merge(num_fires_by_year, total_size_by_year, on='FIRE_YEAR')

print(num_fires_by_year)


    FIRE_YEAR  NUM_FIRES  AVG_DURATION  TOTAL BURN ACRES (000s)
0        1992      67961      1.430787              2199.922531
1        1993      61975      1.338402              2191.587003
2        1994      75932      1.692968              4115.582349
3        1995      71440      1.361221              2049.553158
4        1996      75561      1.506307              6004.833259
5        1997      61442      1.552868              3231.569010
6        1998      68356      1.643028              2015.942300
7        1999      89350      1.706216              6136.947415
8        2000      96396      1.919543              7775.688664
9        2001      87001      1.586502              3836.399064
10       2002      76127      1.775514              6824.142898
11       2003      68275      2.079488              4511.189306
12       2004      69370      1.838690              8248.571174
13       2005      92921      1.833014              9710.848909
14       2006     117943      1.892215  

In [29]:
# Group by MONTH and show number of fires and percentage (across months, for full time series)
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

num_fires_by_month = df.groupby(df['MONTH'])['FIRE_SIZE'].count().reset_index(name='NUM_FIRES')
num_fires_by_month['MONTH'] = pd.Categorical(num_fires_by_month['MONTH'], categories=month_order, ordered=True)
num_fires_by_month['PERCENTAGE'] = (num_fires_by_month['NUM_FIRES'] / num_fires_by_month['NUM_FIRES'].sum()) * 100

# Sort by month
num_fires_by_month = num_fires_by_month.sort_values('MONTH')

# Display the result
print(num_fires_by_month)


   MONTH  NUM_FIRES  PERCENTAGE
4    Jan     114549    4.972686
3    Feb     175843    7.633519
7    Mar     284157   12.335537
0    Apr     285065   12.374955
8    May     203979    8.854931
6    Jun     207004    8.986249
5    Jul     297769   12.926448
1    Aug     250990   10.895725
11   Sep     163292    7.088668
10   Oct     130899    5.682456
9    Nov     118506    5.144463
2    Dec      71511    3.104363


In [30]:
# Group by CAUSE_CLASSIFICATION and show number and percent of fires by cause each year 
fires_by_year_cause = df.groupby(['FIRE_YEAR', 'NWCG_CAUSE_CLASSIFICATION']).size().reset_index(name='NUM_FIRES')

# Pivot the data
pivot_table = fires_by_year_cause.pivot(index='NWCG_CAUSE_CLASSIFICATION', columns='FIRE_YEAR', values='NUM_FIRES').fillna(0)

# Calculate percentages for each year
percentages = pivot_table.div(pivot_table.sum(axis=0), axis=1) * 100

# Combine counts and percentages
pivot_with_percent = pivot_table.astype(int).astype(str) + ' (' + percentages.round(1).astype(str) + '%)'

# Remove index label
pivot_with_percent.index.name = None

print(pivot_with_percent)


FIRE_YEAR               1992           1993           1994           1995  \
Human          47399 (69.7%)  45417 (73.3%)  49617 (65.3%)  55006 (77.0%)   
Natural        12576 (18.5%)   7848 (12.7%)  16878 (22.2%)   8518 (11.9%)   
Not specified   7986 (11.8%)   8710 (14.1%)   9437 (12.4%)   7916 (11.1%)   

FIRE_YEAR               1996           1997           1998           1999  \
Human          53264 (70.5%)  51123 (83.2%)  55778 (81.6%)  75701 (84.7%)   
Natural        13149 (17.4%)   8866 (14.4%)  11356 (16.6%)  12308 (13.8%)   
Not specified   9148 (12.1%)    1453 (2.4%)    1222 (1.8%)    1341 (1.5%)   

FIRE_YEAR               2000           2001  ...           2011  \
Human          78704 (81.6%)  71384 (82.0%)  ...  70107 (70.9%)   
Natural        16907 (17.5%)  14255 (16.4%)  ...  12991 (13.1%)   
Not specified     785 (0.8%)    1362 (1.6%)  ...  15805 (16.0%)   

FIRE_YEAR               2012           2013           2014           2015  \
Human          58434 (78.7%)  50974 

In [56]:
# Group by CAUSE_CLASSIFICATION, and show number and percent of burned acres by cause each year
fires_by_year_cause = df.groupby(['FIRE_YEAR', 'NWCG_CAUSE_CLASSIFICATION'])['FIRE_SIZE'].sum().reset_index(name='TOTAL_FIRE_SIZE')

# Pivot the data
pivot_table = fires_by_year_cause.pivot(index='NWCG_CAUSE_CLASSIFICATION', columns='FIRE_YEAR', values='TOTAL_FIRE_SIZE').fillna(0)

# Calculate percentages for each year based on total fire size
percentages = pivot_table.div(pivot_table.sum(axis=0), axis=1) * 100

# Combine fire sizes and percentages
pivot_with_percent = pivot_table.round(2).astype(str) + ' (' + percentages.round(1).astype(str) + '%)'

# Remove index label
pivot_with_percent.index.name = None

print(pivot_with_percent)


FIRE_YEAR                    1992                1993                1994  \
Human          1012978.51 (46.0%)   815487.43 (37.2%)  1224765.64 (29.8%)   
Natural        1007933.45 (45.8%)  1113048.05 (50.8%)  2397151.24 (58.2%)   
Not specified    179010.57 (8.1%)   263051.52 (12.0%)   493665.46 (12.0%)   

FIRE_YEAR                   1995                1996                1997  \
Human          974605.97 (47.6%)  2101623.19 (35.0%)    831512.7 (25.7%)   
Natural        834573.81 (40.7%)  3324920.17 (55.4%)  2218446.28 (68.6%)   
Not specified  240373.38 (11.7%)     578289.9 (9.6%)    181610.03 (5.6%)   

FIRE_YEAR                   1998                1999               2000  \
Human           983625.3 (48.8%)  2317892.57 (37.8%)  2759928.9 (35.5%)   
Natural        848569.52 (42.1%)  3676823.92 (59.9%)  4940180.5 (63.5%)   
Not specified   183747.48 (9.1%)    142230.92 (2.3%)    75579.27 (1.0%)   

FIRE_YEAR                    2001  ...                2011  \
Human          1768397.

In [31]:
# Group by GENERAL_CAUSE and show number and percent of fires by general cause each year
fires_by_year_general_cause = df.groupby(['FIRE_YEAR', 'NWCG_GENERAL_CAUSE']).size().reset_index(name='NUM_FIRES')

# Pivot the data
pivot_table_general = fires_by_year_general_cause.pivot(index='NWCG_GENERAL_CAUSE', columns='FIRE_YEAR', values='NUM_FIRES').fillna(0)

# Calculate percentages for each year
percentages_general = pivot_table_general.div(pivot_table_general.sum(axis=0), axis=1) * 100

# Combine counts and percentages
pivot_with_percent_general = pivot_table_general.astype(int).astype(str) + ' (' + percentages_general.round(1).astype(str) + '%)'

# Remove index label
pivot_with_percent_general.index.name = None

print(pivot_with_percent_general)


FIRE_YEAR                                            1992           1993  \
Arson/incendiarism                          11250 (16.6%)  11036 (17.8%)   
Debris and open burning                     12792 (18.8%)  12508 (20.2%)   
Equipment and vehicle use                     5637 (8.3%)    5516 (8.9%)   
Firearms and explosives use                      1 (0.0%)       1 (0.0%)   
Fireworks                                      278 (0.4%)     278 (0.4%)   
Misuse of fire by a minor                     2750 (4.0%)    2288 (3.7%)   
Natural                                     12576 (18.5%)   7848 (12.7%)   
Not specified                               14837 (21.8%)  14931 (24.1%)   
Other causes                                   230 (0.3%)     122 (0.2%)   
Power generation/transmission/distribution     192 (0.3%)     166 (0.3%)   
Railroad operations and maintenance           2000 (2.9%)    1840 (3.0%)   
Recreation and ceremony                       2774 (4.1%)    3012 (4.9%)   
Smoking     

In [32]:
# Aggregate GENERAL_CAUSEs by type and show number and percent of fires by general cause type each year
cause_mapping = {
    'Power generation/transmission/distribution': 'Human - accidental',
    'Debris and open burning': 'Human - accidental',
    'Recreation and ceremony': 'Human - accidental',
    'Equipment and vehicle use': 'Human - accidental',
    'Arson/incendiarism': 'Human - arson',
    'Fireworks': 'Human - accidental',
    'Other causes': 'Other',
    'Railroad operations and maintenance': 'Human - accidental',
    'Smoking': 'Human - accidental',
    'Misuse of fire by a minor': 'Human - accidental',
    'Firearms and explosives use': 'Human - accidental',
    'Natural': 'Natural causes',
    'Not specified': 'Not specified'
}

# Apply the mapping to create a new column for aggregated categories
df['AGGREGATED_CAUSE'] = df['NWCG_GENERAL_CAUSE'].map(cause_mapping)

# Group by FIRE_YEAR and AGGREGATED_CAUSE
fires_by_year_agg = df.groupby(['FIRE_YEAR', 'AGGREGATED_CAUSE']).size().reset_index(name='NUM_FIRES')

# Pivot the data
pivot_table_agg = fires_by_year_agg.pivot(index='AGGREGATED_CAUSE', columns='FIRE_YEAR', values='NUM_FIRES').fillna(0)

# Calculate percentages for each year
percentages_agg = pivot_table_agg.div(pivot_table_agg.sum(axis=0), axis=1) * 100

# Combine counts and percentages
pivot_with_percent_agg = pivot_table_agg.astype(int).astype(str) + ' (' + percentages_agg.round(1).astype(str) + '%)'

# Remove index label
pivot_with_percent_agg.index.name = None

print(pivot_with_percent_agg)

FIRE_YEAR                    1992           1993           1994  \
Human - accidental  29068 (42.8%)  28038 (45.2%)  32747 (43.1%)   
Human - arson       11250 (16.6%)  11036 (17.8%)  10235 (13.5%)   
Natural causes      12576 (18.5%)   7848 (12.7%)  16878 (22.2%)   
Not specified       14837 (21.8%)  14931 (24.1%)  15879 (20.9%)   
Other                  230 (0.3%)     122 (0.2%)     193 (0.3%)   

FIRE_YEAR                    1995           1996           1997  \
Human - accidental  35295 (49.4%)  36616 (48.5%)  31084 (50.6%)   
Human - arson       12889 (18.0%)  10449 (13.8%)  11335 (18.4%)   
Natural causes       8518 (11.9%)  13149 (17.4%)   8866 (14.4%)   
Not specified       14443 (20.2%)  15094 (20.0%)   9877 (16.1%)   
Other                  295 (0.4%)     253 (0.3%)     280 (0.5%)   

FIRE_YEAR                    1998           1999           2000  \
Human - accidental  33113 (48.4%)  45164 (50.5%)  42946 (44.6%)   
Human - arson       14199 (20.8%)  19587 (21.9%)  18040 (18.

In [58]:
# print(df['STATE'].value_counts().sort_index())
# print('Number of states and territories:', df['STATE'].nunique())

GACC_region_map = {
    'AK': 'Alaska',
    'AL': 'Southern Area',
    'AR': 'Southern Area',
    'AZ': 'Southwest',
    'CA': 'California',
    'CO': 'Rocky Mountain',
    'CT': 'Eastern Area',
    'DC': 'Eastern Area',
    'DE': 'Eastern Area',
    'FL': 'Southern Area',
    'GA': 'Southern Area',
    'HI': 'Hawaii',
    'IA': 'Eastern Area',
    'ID': 'Great Basin',
    'IL': 'Eastern Area',
    'IN': 'Eastern Area',
    'KS': 'Rocky Mountain',
    'KY': 'Southern Area',
    'LA': 'Southern Area',
    'MA': 'Eastern Area',
    'MD': 'Eastern Area',
    'ME': 'Eastern Area',
    'MI': 'Eastern Area',
    'MN': 'Eastern Area',
    'MO': 'Eastern Area',
    'MS': 'Southern Area',
    'MT': 'Northern Rockies',
    'NC': 'Southern Area',
    'ND': 'Northern Rockies',
    'NE': 'Rocky Mountain',
    'NH': 'Eastern Area',
    'NJ': 'Eastern Area',
    'NM': 'Southwest',
    'NV': 'Great Basin',
    'NY': 'Eastern Area',
    'OH': 'Eastern Area',
    'OK': 'Southern Area',
    'OR': 'Northwest',
    'PA': 'Eastern Area',
    'PR': 'Southern Area',
    'RI': 'Eastern Area',
    'SC': 'Southern Area',
    'SD': 'Rocky Mountain',
    'TN': 'Southern Area',
    'TX': 'Southern Area',
    'UT': 'Great Basin',
    'VA': 'Southern Area',
    'VT': 'Eastern Area',
    'WA': 'Northwest',
    'WI': 'Eastern Area',
    'WV': 'Eastern Area',
    'WY': 'Rocky Mountain'
}

# Group by region without adding column, report number and percent of fires per region per year
result = df.groupby([df['STATE'].map(GACC_region_map), 'FIRE_YEAR']).size().unstack(fill_value=0)

# Calculate percentages within each year
percentages = result.div(result.sum(axis=0), axis=1) * 100

# Format the output with counts and percentages
formatted = result.astype(str) + ' (' + percentages.round(1).astype(str) + '%)'

print(formatted.rename_axis(None, axis=0))


FIRE_YEAR                  1992           1993           1994           1995  \
Alaska               480 (0.7%)     859 (1.4%)     639 (0.8%)     414 (0.6%)   
California        10831 (15.9%)   8268 (13.3%)   8649 (11.4%)   7381 (10.3%)   
Eastern Area       8687 (12.8%)   6451 (10.4%)   9750 (12.8%)  11149 (15.6%)   
Great Basin         3843 (5.7%)    1851 (3.0%)    5709 (7.5%)    3128 (4.4%)   
Hawaii                77 (0.1%)      14 (0.0%)       9 (0.0%)      10 (0.0%)   
Northern Rockies    2036 (3.0%)    1129 (1.8%)    3480 (4.6%)    1324 (1.9%)   
Northwest           5730 (8.4%)    2838 (4.6%)    5494 (7.2%)    3477 (4.9%)   
Rocky Mountain      2662 (3.9%)    1982 (3.2%)    4443 (5.9%)    2798 (3.9%)   
Southern Area     29144 (42.9%)  32139 (51.9%)  30572 (40.3%)  35593 (49.8%)   
Southwest           4471 (6.6%)   6444 (10.4%)    7187 (9.5%)    6166 (8.6%)   

FIRE_YEAR                  1996           1997           1998           1999  \
Alaska               718 (1.0%)     720

In [68]:
# report number and percent of fires per region per month, distribution across regions within a month

# Define month order
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Group by region and month
result = df.groupby([df['STATE'].map(GACC_region_map), 'MONTH']).size().unstack(fill_value=0)

# Reorder columns by month
result = result[month_order]

# Calculate percentages within each month
percentages = result.div(result.sum(axis=0), axis=1) * 100

# Format the output with counts and percentages
formatted = result.astype(str) + ' (' + percentages.round(1).astype(str) + '%)'

# Print with region labels but no axis label
print(formatted.rename_axis(None, axis=0))

MONTH                       Jan             Feb             Mar  \
Alaska                 9 (0.0%)       13 (0.0%)      132 (0.0%)   
California          4810 (4.2%)     4417 (2.5%)     5762 (2.0%)   
Eastern Area        6113 (5.3%)    12714 (7.2%)   47248 (16.6%)   
Great Basin          195 (0.2%)      296 (0.2%)     1106 (0.4%)   
Hawaii               752 (0.7%)      496 (0.3%)      513 (0.2%)   
Northern Rockies     293 (0.3%)      434 (0.2%)     2854 (1.0%)   
Northwest            180 (0.2%)      420 (0.2%)     1420 (0.5%)   
Rocky Mountain      5213 (4.6%)     7017 (4.0%)    17180 (6.0%)   
Southern Area     92639 (80.9%)  145107 (82.5%)  199832 (70.3%)   
Southwest           4345 (3.8%)     4929 (2.8%)     8110 (2.9%)   

MONTH                        Apr            May            Jun            Jul  \
Alaska               1053 (0.4%)    4053 (2.0%)    4708 (2.3%)    3389 (1.1%)   
California          11212 (3.9%)  25027 (12.3%)  40159 (19.4%)  53011 (17.8%)   
Eastern Area      1

In [69]:
# report number and percent of fires per region per month, distribution across months within a region

# Group by region and month
result = df.groupby([df['STATE'].map(GACC_region_map), 'MONTH']).size().unstack(fill_value=0)

# Reorder columns by month
result = result[month_order]

# Transpose the table so months are rows and regions are columns
result = result.T

# Calculate percentages across months for each region (each column sums to 100%)
percentages = result.div(result.sum(axis=0), axis=1) * 100

# Format the output with counts and percentages
formatted = result.astype(str) + ' (' + percentages.round(1).astype(str) + '%)'

# Display result with months as rows, regions as columns, and no extra axis label
print(formatted.rename_axis(None, axis=0))



STATE        Alaska     California    Eastern Area    Great Basin  \
Jan        9 (0.1%)    4810 (1.9%)     6113 (1.7%)     195 (0.2%)   
Feb       13 (0.1%)    4417 (1.8%)    12714 (3.5%)     296 (0.3%)   
Mar      132 (0.9%)    5762 (2.3%)   47248 (13.1%)    1106 (1.1%)   
Apr     1053 (6.9%)   11212 (4.5%)  107608 (29.9%)    1856 (1.9%)   
May    4053 (26.7%)   25027 (9.9%)   56021 (15.6%)    4058 (4.1%)   
Jun    4708 (31.0%)  40159 (15.9%)    21980 (6.1%)  12354 (12.4%)   
Jul    3389 (22.3%)  53011 (21.0%)    25658 (7.1%)  29962 (30.2%)   
Aug     1138 (7.5%)  41887 (16.6%)    19807 (5.5%)  32819 (33.1%)   
Sep      529 (3.5%)  30707 (12.2%)    15987 (4.4%)  11937 (12.0%)   
Oct      131 (0.9%)   20178 (8.0%)    16778 (4.7%)    3728 (3.8%)   
Nov       26 (0.2%)    9711 (3.9%)    23382 (6.5%)     782 (0.8%)   
Dec       14 (0.1%)    5000 (2.0%)     6181 (1.7%)     158 (0.2%)   

STATE        Hawaii Northern Rockies      Northwest Rocky Mountain  \
Jan      752 (7.5%)       293 (0

In [57]:
df.to_csv('fire_data_cleaned_bp.csv')
print("CSV output file created!")

CSV output file created!


In [58]:
# Convert DataFrame to JSON
json_data = df.to_json(orient='records', indent=4)

# Save JSON to a file
with open('fire_data_cleaned_bp.json', 'w') as f:
    f.write(json_data)

print("CSV successfully converted to JSON!")

CSV successfully converted to JSON!
